In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import time

# Step 1: Scraping Drug Data

def scrape_drugs():
    base_url = "https://www.drugs.com/drug_information.html"
    response = requests.get(base_url)
    soup = BeautifulSoup(response.content, "html.parser")

    drug_list_section = soup.find("ul", {"class": "ddc-list-column-4"})
    if not drug_list_section:
        print("Drug list section not found.")
        return pd.DataFrame()

    drug_links = [link["href"] for link in drug_list_section.find_all("a", href=True)[:50]]
    drugs_data = []
    for link in drug_links:
        drug_url = f"https://www.drugs.com{link}"
        drug_info = scrape_drug_info(drug_url)
        drugs_data.append(drug_info)
        time.sleep(1)  # Avoid overloading the server
    return pd.DataFrame(drugs_data)

def scrape_drug_info(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    drug_name = soup.find("h1").text.strip() if soup.find("h1") else "N/A"
    uses_tag = soup.find("h2", string=lambda text: text and "What is" in text)
    uses = uses_tag.find_next("p").text.strip() if uses_tag and uses_tag.find_next("p") else "N/A"
    side_effects = []
    side_effects_tag = soup.find("h2", id="side-effects")
    if side_effects_tag:
        element = side_effects_tag.find_next_sibling()
        while element:
            if element.name == "h2":
                break
            side_effects.append(element.text.strip())
            element = element.find_next_sibling()
    side_effects = ", ".join(side_effects) if side_effects else "N/A"
    return {"Drug": drug_name, "Uses": uses, "SideEffect": side_effects}

# Step 2: Cleaning Data

def clean_data(df):
    df["Drug"] = df["Drug"].str.replace(r"[^a-zA-Z0-9\s]", "", regex=True).str.strip()
    df["Uses"] = df["Uses"].str.lower()
    df["SideEffect"] = df["SideEffect"].str.lower()
    df.drop_duplicates(subset=["Drug"], inplace=True)
    return df

# Step 3: SQL Analysis

DATABASE = "drug_data.db"

def save_to_sqlite(df):
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()
    # Create the table if it doesn't exist
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS drugs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            Drug TEXT NOT NULL,
            Uses TEXT NOT NULL,
            SideEffect TEXT
        );
    ''')
    conn.commit()

    # Insert data into the table
    df.to_sql("drugs", conn, if_exists="replace", index=False)
    conn.close()

def perform_sql_analysis():
    conn = sqlite3.connect(DATABASE)

    # 1. Top 5 Most Common Uses
    top_uses_query = """
    SELECT Uses, COUNT(*) AS Frequency
    FROM drugs
    WHERE Uses != 'n/a'
    GROUP BY Uses
    ORDER BY Frequency DESC
    LIMIT 5;
    """
    top_uses = pd.read_sql_query(top_uses_query, conn)
    print("Top 5 Most Common Uses:")
    print(top_uses)

    # 2. Most Frequent Side Effect
    side_effect_query = """
    WITH SplitSideEffects AS (
        SELECT TRIM(value) AS SideEffect
        FROM json_each(SideEffect)  
        WHERE value IS NOT NULL
    )
    SELECT SideEffect, COUNT(*) AS Frequency
    FROM SplitSideEffects
    WHERE SideEffect != 'n/a'
    GROUP BY SideEffect
    ORDER BY Frequency DESC
    LIMIT 1;
    """
    try:
        most_frequent_side_effect = pd.read_sql_query(side_effect_query, conn)
        if not most_frequent_side_effect.empty:
            print("\nMost Frequent Side Effect:")
            print(most_frequent_side_effect)
        else:
            print("\nMost Frequent Side Effect: None found.")
    except Exception as e:
        print(f"Side effect query failed: {e}")

    # 3. Count of Unique Drug Names
    unique_drug_count_query = "SELECT COUNT(DISTINCT Drug) AS UniqueDrugCount FROM drugs;"
    unique_drug_count = pd.read_sql_query(unique_drug_count_query, conn)
    print("\nCount of Unique Drug Names:")
    print(f"Total Unique Drugs: {unique_drug_count['UniqueDrugCount'][0]}")

    conn.close()

# Main execution 1
if __name__ == "__main__":
    print("Scraping drug data...")
    raw_data = scrape_drugs()
    print("Cleaning drug data...")
    cleaned_data = clean_data(raw_data)
    print("Saving data to SQLite database...")
    save_to_sqlite(cleaned_data)
    print("Performing SQL analysis...")
    perform_sql_analysis()


Scraping drug data...
Cleaning drug data...
Saving data to SQLite database...
Performing SQL analysis...
Top 5 Most Common Uses:
                                                Uses  Frequency
0  naltrexone is a prescription medication used t...          1
1  mounjaro (tirzepatide) is used for type 2 diab...          1
2  metoprolol is a beta-blocker that affects the ...          1
3  methotrexate interferes with the growth of cer...          1
4  methadone is a long-acting opioid medication t...          1

Most Frequent Side Effect: None found.

Count of Unique Drug Names:
Total Unique Drugs: 50
